
---

### 🧠 Additive Attention (Bahdanau Attention) 

- Developed to overcome the fixed-length context bottleneck in encoder-decoder RNN/LSTM models.
- Instead of relying on a single vector for the entire input sequence, the decoder **attends to all encoder hidden states** while generating each output token.
- For each output time step \( i \), the decoder computes a **context vector** \( \mathbf{c}_i \) as a **weighted sum of encoder annotations** \( \mathbf{h}_j \):

  $$
  \mathbf{c}_i = \sum_{j=1}^{T_x} \alpha_{ij} \mathbf{h}_j \tag{5}
  $$

- The attention weights \( \alpha_{ij} \) reflect the relevance of each encoder hidden state \( \mathbf{h}_j \) to the current decoder state \( \mathbf{s}_{i-1} \):

  $$
  \alpha_{ij} = \frac{\exp(e_{ij})}{\sum_{k=1}^{T_x} \exp(e_{ik})} \tag{6}
  $$

- The **alignment score** \( e_{ij} \) is computed using a small feedforward neural network (the **alignment model**) that takes \( \mathbf{s}_{i-1} \) and \( \mathbf{h}_j \) as inputs:

  $$
  e_{ij} = \mathbf{v}^T \tanh(\mathbf{W}_1 \mathbf{s}_{i-1} + \mathbf{W}_2 \mathbf{h}_j + \mathbf{b}) \tag{7}
  $$

  - v: learnable weight vector
  - W_1, W_2: learnable weight matrices
  - b : bias term

- This is called **additive attention** because the alignment model uses **addition** (not dot product) to combine the decoder and encoder hidden states.

- The attention mechanism allows the decoder to dynamically **focus on relevant parts** of the input sequence during each generation step.

---



---

### ✅ In **Dot-Product Attention (Luong)** used in RNN-based encoder-decoder:

- The **basic variant** computes the score as:
  $$
  e_{ij} = \mathbf{s}_i^\top \mathbf{h}_j
  $$
  where:
  - s_i: decoder hidden state at time step \( i \)
  - h_j: encoder hidden state at time step \( j \)

- This is **just a dot product** — **no learnable parameters** involved in this scoring step.

---

### Why use dot-product attention then?

- ✅ **Efficiency**: It’s faster — no extra parameters or matrix ops.
- ❌ **Less expressive**: Cannot learn complex alignments or transformations.

---



---

### 🔁 Limitation in RNN-based Attention Mechanisms (Both Additive & Dot-Product)

- In both **additive attention (Bahdanau)** and **dot-product attention (Luong)**, attention is computed **from the decoder to the encoder**:
  - The decoder focuses on different encoder hidden states while generating each output.
  - But there is **no attention among encoder tokens themselves**.
  
- ⚠️ That means:
  - **Input tokens do not attend to each other.**
  - The encoder processes the input sequentially using RNNs (or LSTMs), and the final hidden states are used in attention.
  - This limits the model’s ability to capture **long-range dependencies** and **global interactions** within the input sequence.

- ✅ This limitation was addressed by the **Transformer model** (Vaswani et al., 2017), which introduced:
  - **Self-attention**: each token in the input sequence **attends to all other tokens**, capturing global context.
  - **Positional encoding**: to retain sequence order, since self-attention is not sequential like RNNs.

---



---

### 🔍 Why RNNs (even Bidirectional) Still Struggled to Capture Global Token Relations

#### ✅ What Bidirectional RNNs *do*:
- A **unidirectional RNN/LSTM** only sees the past: for time step \( t \), it processes from \( x_1 \) to \( x_t \).
- A **bidirectional RNN (BiRNN)** adds another RNN that processes in reverse: from \( x_T \) to \( x_1 \).
- So at each time step \( t \), the hidden state \( h_t \) contains information from both past and future tokens — i.e., **local context** from both directions.

#### ❌ What they *still can't do well*:
- Even with both directions, **each hidden state still represents only a compressed summary** of the context — and that compression is:
  - **Sequential**
  - **Fixed-size**
  - **Hard to optimize over long sequences**
- **No direct interaction** between distant tokens unless the information is passed **step-by-step**, which leads to:
  - **Vanishing gradients**
  - **Poor long-range dependency modeling**
- For example, the relationship between \( x_3 \) and \( x_{98} \) has to be encoded **indirectly** through dozens of hidden states in between.

---

### 🤝 Why Attention Helped:
- Attention allows the model to **look at all tokens at once**, not just rely on the final compressed hidden state.
- But in **encoder-decoder attention**, the **input tokens (encoder hidden states) are never allowed to look at each other**.
  - The decoder attends to the encoder outputs.
  - But **encoder tokens don’t attend to other encoder tokens** — each hidden state is still generated sequentially.

---

### 🔁 Transformers: Game-Changer
- Introduced **self-attention** in both encoder and decoder.
- Now, **every token attends to every other token directly** (not through hidden states).
- This enables the model to:
  - Capture **global dependencies directly**
  - Be **parallelizable** (no need for sequential processing)
  - Better handle **long-range interactions**

---

### 🔑 TL;DR:
> Even bidirectional RNNs process sequences sequentially. They encode context *around* a token, but **don't model direct pairwise interactions** between all tokens. That’s why they can miss global relationships — especially over long distances. Attention fixed this for decoder → encoder, and Transformers fixed it fully with **self-attention**.

---



---

## 🔁 Self-Attention & Contextualized Embeddings

- **Self-attention** (or **intra-attention**) is a mechanism where:
  - Each token in a sequence **attends to all other tokens**, including itself.
  - It calculates **how much focus** to place on other tokens when updating its own representation.

- The result is a **new embedding** for each token — called a **contextualized embedding**.

- Unlike static embeddings (e.g., Word2Vec, GloVe), which assign **one fixed vector per word**, self-attention allows the **same word to have different embeddings depending on context**.

---

## 🧠 Why Context Matters — The “Apple” Example

Imagine these two sentences:

1. **"I ate a juicy apple after lunch."** 🍎  
2. **"I updated my Apple phone to the latest version."** 📱

- In both cases, the word **"apple"** is spelled the same.
- But their **meanings are completely different** — fruit vs tech company.

---

### ✅ How Self-Attention Helps:

| Sentence                              | Context Around "Apple"         | Resulting Embedding          |
|---------------------------------------|--------------------------------|------------------------------|
| "I ate a juicy **apple** after lunch" | Tokens like *ate*, *juicy*, *lunch* | Embedding leans toward **fruit** 🍏 |
| "I updated my **Apple** phone..."     | Tokens like *updated*, *phone*, *version* | Embedding leans toward **technology** 📱 |

- Thanks to **self-attention**, "apple" attends to words around it.
- The model **learns the meaning from context**, so the embedding for "apple" becomes **context-aware**.

---

## 🎯 Key Benefits of Contextualized Embeddings:

- 🔍 **Disambiguation**: Helps models distinguish between multiple meanings of the same word.
- 🧠 **Rich understanding**: Better grasp of semantics and syntax.
- 💬 **Improved performance**: Boosts accuracy in downstream tasks like translation, sentiment analysis, QA, etc.

---



---

## ✅ Multi-Head Self-Attention

Let’s assume:

- **Embedding dimension** = 512  
- **Number of heads** = 8  
- Therefore, each head operates on **512 / 8 = 64** dimensions  

---

### 🔢 Step-by-Step Breakdown

#### 1. **Start with token input:**
- You begin with the **input embedding**:
  $$
  \text{input} = \text{token embedding} + \text{positional encoding} \in \mathbb{R}^{512}
  $$

---

#### 2. **Linear projections per head:**
- We **project it into separate Q, K, V vectors for each head** using **separate weight matrices** for each head.
  
- So for **each head \( i \in [1, 8] \)**:
  $$
  Q^{(i)} = X W_Q^{(i)}, \quad K^{(i)} = X W_K^{(i)}, \quad V^{(i)} = X W_V^{(i)}
  $$
  
  where:
  -  $$W_Q^{(i)}, W_K^{(i)}, W_V^{(i)} \in \mathbb{R}^{512 \times 64}$$ 
  -  $$Q^{(i)}, K^{(i)}, V^{(i)} \in \mathbb{R}^{n \times 64}$$ 
   for sequence length n 

✅ So yes: each matrix maps the **full 512-dim input** to a **64-dim subspace** for that head.

---

#### 3. **Self-attention for each head:**
- Each head computes scaled dot-product attention **independently**:
  $$
  \text{Attention}^{(i)} = \text{softmax} \left( \frac{Q^{(i)} {K^{(i)}}^\top}{\sqrt{64}} \right) V^{(i)}
  $$

---

#### 4. **Concatenation of heads:**
- After computing attention outputs for all 8 heads (each of size \( \mathbb{R}^{n \times 64} \)), you **concatenate them** along the feature dimension:
  $$
  \text{Concat} = [\text{head}_1; \text{head}_2; \dots; \text{head}_8] \in \mathbb{R}^{n \times 512}
  $$

---

#### 5. **Final linear projection (W₀):**
- You apply a final learned linear layer:
  $$
  \text{Output} = \text{Concat} \cdot W_O
  $$
  where \( W_O \in \mathbb{R}^{512 \times 512} \)

- This projects the concatenated output **back to the original embedding dimension (512)**.

---


---

## 🧠 Time Complexity of Self-Attention

Let:
- \( n \): sequence length
- \( d \): embedding dimension
- \( h \): number of heads
- \( d_k = d/h \): dimension per head (usually, \( d_k \approx 64 \))

---

### 🔁 1. **Self-Attention (Single Head)**

Self-attention requires the following operations per layer:

#### 📌 Step-by-step breakdown:

| Operation | Complexity |
|----------|------------|
| **Linear projections** (Q, K, V from input \( X \in \mathbb{R}^{n \times d} \)) | \( O(n \cdot d^2) \) |
| **Dot product attention (QKᵀ)** | \( O(n^2 \cdot d) \) |
| **Softmax over \( n \) tokens** | \( O(n^2) \) |
| **Multiply attention weights with V** | \( O(n^2 \cdot d) \) |

✅ **Total (Single-Head)**:  
$$
\boxed{O(n^2 \cdot d + n \cdot d^2)}
$$

- \( O(n^2 \cdot d) \): due to pairwise interactions for attention
- \( O(n \cdot d^2) \): due to the linear projections (Q, K, V)

> The dominating term is usually **\( O(n^2 \cdot d) \)** due to the attention matrix \( QK^\top \).

---

### 🔀 2. **Multi-Head Attention**

In multi-head attention, we perform the attention operation **in parallel for each head**, with smaller dimensions \( d_k = d/h \), and then concatenate.

So:
- **Per head**: \( O(n^2 \cdot d_k + n \cdot d \cdot d_k) \)
- Across **\( h \)** heads: \( h \cdot O(n^2 \cdot d_k) = O(n^2 \cdot d) \)

Final projection (concatenated heads back to \( d \)-dim):
- \( O(n \cdot d^2) \)

✅ **Total (Multi-Head)**:  
$$
\boxed{O(n^2 \cdot d + n \cdot d^2)}
$$

> Same as single-head in big-O — but **more efficient in practice due to parallelism**.

---

## ⚠️ Bottleneck Insight

- **Quadratic time complexity in sequence length \( n \)**:  
  - \( O(n^2 \cdot d) \) → the reason why Transformers become slow for long sequences (e.g. long documents or audio).
  - This comes from computing \( QK^\top \in \mathbb{R}^{n \times n} \).

---

## 🧪 Summary Table

| Mechanism          | Time Complexity               | Notes                        |
|--------------------|-------------------------------|------------------------------|
| Self-Attention     | \( O(n^2 \cdot d + n \cdot d^2) \) | Quadratic in \( n \), due to token-token interactions |
| Multi-Head Attention | \( O(n^2 \cdot d + n \cdot d^2) \) | Same complexity, but more powerful with multiple subspaces |

---

Let me know if you want a visual to show how this scales or want a comparison with **Recurrent** or **Convolutional** layers!

## What is the need of scaling with the 1/sqrt(d_k) why didn't we use some other number instead of d_k